In [1]:
from data_generator.csv_data_processor import CSVDataProcessor

In [3]:
from utils.util_funcs import get_row_count, generate_dob

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL").getOrCreate()

csv_reader = CSVDataProcessor(spark, "data/healthcare_dataset.csv")

# Read the CSV file
df = csv_reader.run()

# Show DataFrame
df.show()
df.createOrReplaceTempView("raw_healthcare_data")
get_row_count(df)


24/01/22 20:58:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


runner func.


+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|                name|age|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|
+--------------------+---+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|     Tiffany Ramirez| 81|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|
|         Ruben Burns| 35|  Male|        O+|           Asthma|       2023-06-01|     Diane Jacks

In [4]:
# distinct_docs = spark.sql(f"select distinct Doctor from raw_healthcare_data")

In [5]:
# distinct_docs.show()

In [6]:
# distinct_patients_age = spark.sql("SELECT DISTINCT Name, Age FROM raw_healthcare_data")
# distinct_patients_name = spark.sql("SELECT DISTINCT Name FROM raw_healthcare_data")

In [7]:
# get_row_count(distinct_patients_name)
# get_row_count(distinct_patients_age)


In [8]:
# admission_types = spark.sql("SELECT DISTINCT Admission_Type FROM raw_healthcare_data")

In [9]:
# get_row_count(admission_types)

In [10]:
new_admission_types = ["emergency", "gp_referral", "hospital_referral", "self_referral", "elective"]
sub_admission_types = [
    "maternity", "mental", "injury_rtc", "urgent", "self_inflicted", "cancer",
    "neurology", "cardiology", "orthopedics", "pediatrics", "gastroenterology",
    "pulmonology", "nephrology", "endocrinology", "dermatology", "oncology",
    "ophthalmology", "otorhinolaryngology(ENT)", "geriatrics", "obstetrics", "infectious_diseases"
]
stay_type = ["outpatient", "inpatient", "day_patient"]

sub_admission_tests = {
    "maternity": ["Ultrasound", "Blood tests", "Glucose tolerance test", "Amniocentesis"],
    "mental": ["Psychological evaluation", "Blood tests", "Brain imaging (MRI, CT scans)", "Electroencephalogram (EEG)"],
    "injury_rtc": ["X-rays", "CT scans", "MRI", "Ultrasound", "Blood tests"],
    "urgent": ["Varies by condition", "Blood tests", "X-rays", "ECG", "CT scans", "MRI"],
    "self_inflicted": ["Psychological assessment", "X-rays (for physical injuries)", "Blood tests", "Toxicology screening"],
    "cancer": ["Biopsy", "Blood tests", "MRI", "CT scans", "PET scans", "X-rays", "Ultrasound"],
    "neurology": ["MRI or CT scans of the brain", "Electroencephalogram (EEG)", "Lumbar puncture", "Nerve conduction studies", "Blood tests"],
    "cardiology": ["ECG", "Echocardiogram", "Stress tests", "Cardiac catheterization", "Blood tests"],
    "orthopedics": ["X-rays", "MRI", "CT scans", "Bone scans", "Blood tests"],
    "pediatrics": ["Blood tests", "Urine tests", "X-rays", "Ultrasound", "MRI", "Developmental screening tests"],
    "gastroenterology": ["Endoscopy", "Colonoscopy", "Blood tests", "Stool tests", "Abdominal ultrasound", "CT scan"],
    "pulmonology": ["Pulmonary function tests", "Chest X-ray", "CT scan", "Bronchoscopy"],
    "nephrology": ["Blood tests (renal function tests)", "Urine tests", "Ultrasound of the kidneys", "Biopsy"],
    "endocrinology": ["Blood tests (hormone levels)", "Thyroid function tests", "Bone density tests"],
    "dermatology": ["Skin biopsy", "Patch tests", "Skin scrapings", "Blood tests"],
    "oncology": ["Biopsies", "Blood tests", "Imaging tests (CT, MRI, PET scans)"],
    "ophthalmology": ["Eye exam", "Tonometry (eye pressure test)", "Retinal imaging", "Visual field test"],
    "otorhinolaryngology": ["Hearing tests", "Endoscopy of the ear/nose/throat", "Imaging tests (CT or MRI)"],
    "geriatrics": ["Comprehensive geriatric assessment", "Blood tests", "Bone density scans", "Cognitive tests"],
    "obstetrics": ["Ultrasound", "Blood tests", "Glucose tolerance test", "Amniocentesis", "Cervical screening"],
    "infectious_diseases": ["Blood cultures", "PCR tests", "Antibody tests", "Imaging (e.g., chest X-ray for pneumonia)", "Lumbar puncture"]
}


In [11]:
df_with_dob = df.withColumn("DOB", generate_dob()).drop("age")


In [12]:
df_with_dob.show()

+--------------------+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------+
|                name|gender|blood_type|medical_condition|date_of_admission|            doctor|            hospital|insurance_provider|    billing_amount|room_number|admission_type|discharge_date| medication|test_results|     DOB|
+--------------------+------+----------+-----------------+-----------------+------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+--------+
|     Tiffany Ramirez|Female|        O-|         Diabetes|       2022-11-17|    Patrick Parker|    Wallace-Hamilton|          Medicare| 37490.98336352819|        146|      Elective|    2022-12-01|    Aspirin|Inconclusive|19580204|
|         Ruben Burns|  Male|        O+|           Asthma|       2023-06-01|

In [96]:
import random
from datetime import datetime
from constants.admission_types_tests_dataset import new_admission_types, admission_mapping, stay_type, admission_tests

def select_random_tests(sub_admission_type):
    tests = admission_tests.get(sub_admission_type, [])
    number_of_tests = random.randint(1, len(tests))  # Randomly choose how many tests to include
    return random.sample(tests, number_of_tests)  # Randomly select tests


def create_new_admission_data(gender: str, dob: str):
    def select_sub_admission():
        return random.choice(list(sub_admission_dict.keys()))
    
    female_only = ["maternity", "obstetrics"]
    
    # calculate if a pediatric case
    today = datetime.now()
    dob_date = datetime.strptime(dob, "%Y%m%d")
    age = today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))
    
    ped_case = "pediatrics" if age < 18 else ''
    
    print('CHILD ', ped_case)
    selected_new_admission_type = random.choice(list(admission_mapping.keys()))
    sub_admission_dict = admission_mapping[selected_new_admission_type]
    selected_sub_admission_type = select_sub_admission()
    if gender.lower() == "male":
        while selected_sub_admission_type in female_only:
            print('CHOOSE AGAIN!')
            selected_sub_admission_type = select_sub_admission()  
            print(selected_sub_admission_type)
    if gender.lower() == 'female':
        print('FEMALE')
        while selected_sub_admission_type == female_only[0] and age < 16:
            print('TOO YOUNG')
            selected_sub_admission_type = select_sub_admission()
            break
    selected_stay_type = random.choice(list(sub_admission_dict[selected_sub_admission_type]))
    selected_tests = select_random_tests(selected_sub_admission_type)
    
    print(f"Selected Admission Type: {selected_new_admission_type}")
    print(f"Selected Sub-Admission Type: {selected_sub_admission_type}")
    print(f"Selected Stay Type: {selected_stay_type}")
    print(f"Selected Tests: {selected_tests}")

In [97]:
create_new_admission_data("female", "20200302")

CHILD  pediatrics
FEMALE
Selected Admission Type: gp_referral
Selected Sub-Admission Type: endocrinology
Selected Stay Type: outpatient
Selected Tests: ['Bone density tests', 'Thyroid function tests']
